# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [1]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api

import ee
import math

# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


Earth Engine initialized.


In [9]:
# 2. Define parameters: bounding box, date range, sunglint threshold, and cloud filter

# -------------------------------------------------------------------
# Modify these variables as needed:
xmin, ymin, xmax, ymax = -80.0, 20.0, -75.0, 25.0  # Example bbox [lon_min, lat_min, lon_max, lat_max]
start_date = '2021-06-01'
end_date   = '2021-08-31'

sun_glint_threshold = 30  # Degrees; scenes with glint angle ≤ this will be selected.
max_cloud_cover = 20      # Maximum cloud percentage for initial filter (0-100)
# -------------------------------------------------------------------

# Create an Earth Engine geometry for the bounding box
bbox = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])


In [10]:
# 3. Load Sentinel-2 L1C collection, filter by date, bounds, and cloud cover

s2_collection = (ee.ImageCollection('COPERNICUS/S2')
                 .filterDate(start_date, end_date)
                 .filterBounds(bbox)
                 .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)))

print('Total S2 scenes in date/area filter:', s2_collection.size().getInfo())


Total S2 scenes in date/area filter: 667


In [ ]:
# 4. Define a function to compute and add the sun glint angle band

def add_sun_glint_angle(img):
    # Convert metadata to radians
    sz = ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE')).multiply(math.pi/180)
    saz = ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).multiply(math.pi/180)
    vz = ee.Number(img.get('MEAN_VIEWING_ZENITH_ANGLE')).multiply(math.pi/180)
    vaz = ee.Number(img.get('MEAN_VIEWING_AZIMUTH_ANGLE')).multiply(math.pi/180)

    # Compute cosines
    cos_sum = sz.add(vz).cos()
    cos_diff = sz.subtract(vz).cos()

    # term1 and term2 for the formula
    term1 = cos_sum.add(cos_diff).divide(2)
    term2 = cos_sum.subtract(cos_diff).divide(2).multiply(saz.subtract(vaz).cos())

    # Sun glint angle in radians
    alpha = term1.add(term2).acos()

    # Convert to degrees
    alpha_deg = alpha.multiply(180/math.pi)

    # Create a constant image from alpha_deg, matching img's projection
    alpha_img = ee.Image.constant(alpha_deg).rename('sunGlintAngle')                    .reproject(crs=img.select('B4').projection(), scale=20)
    return img.addBands(alpha_img)

# Map over the collection
s2_with_glint = s2_collection.map(add_sun_glint_angle)
print('Added sunGlintAngle band to collection.')


Added sunGlintAngle band to collection.


In [23]:
s2_with_glint.size().getInfo()

667

In [24]:
# 5. Filter images where the sun glint angle ≤ threshold

s2_glint_scenes = s2_with_glint.filter(ee.Filter.lte('sunGlintAngle', sun_glint_threshold))
print('Number of glint‐eligible scenes:', s2_glint_scenes.size().getInfo())


Number of glint‐eligible scenes: 0


In [25]:
# 6. (Optional) Mask out non-water pixels using JRC Surface Water dataset
# Load JRC Global Surface Water Occurrence band (0-100%)
jrc = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('occurrence')
water_mask = jrc.gt(50)  # Keep pixels that are water >50% of the year

def mask_non_water(img):
    return img.updateMask(water_mask)

s2_glint_water = s2_glint_scenes.map(mask_non_water)
print('Applied water mask: resulting scenes:', s2_glint_water.size().getInfo())


Applied water mask: resulting scenes: 0


In [26]:
# 7. Print metadata of a few sample glint scenes

sample_list = s2_glint_water.toList(3)
for i in range(3):
    img = ee.Image(sample_list.get(i))
    info = img.getInfo()  # Metadata for inspection
    print(f"Scene {i+1} ID: {info['id']}")
    glint_stats = img.select('sunGlintAngle').reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=bbox,
        scale=20
    )
    print('  sunGlintAngle (deg) min,max:', glint_stats.getInfo())


EEException: List.get: List is empty (index is 0).

In [ ]:
# 8. Compute ΔR_MBSP = (c * B12 − B11) / B11 for each glint scene

def compute_deltaR(img):
    # Mask clouds using QA60 (bit 10): clouds
    cloud_mask = img.select('QA60').bitwiseAnd(1 << 10).eq(0)
    img_masked = img.updateMask(cloud_mask)

    # Perform regression: R11 ≈ c * R12
    reg = img_masked.select(['B12', 'B11']).reduceRegion(
        reducer=ee.Reducer.linearRegression(numX=1, numY=1),
        geometry=bbox,
        scale=20,
        maxPixels=1e9
    )
    coef = ee.Array(reg.get('coefficients')).get([0,0])

    # Compute ΔR_MBSP
    delta = img_masked.expression(
        '(c * b12 - b11) / b11', {
            'c': coef,
            'b12': img_masked.select('B12'),
            'b11': img_masked.select('B11')
        }
    ).rename('deltaR_MBSP')

    return img_masked.addBands(delta)

s2_with_delta = s2_glint_water.map(compute_deltaR)
print('Computed deltaR_MBSP for each glint+water scene.')


Computed deltaR_MBSP for each glint+water scene.
